In [1]:
import Arrow
using DataFrames, TypedTables
using Printf

In [2]:
identifier = "20220208_no_qual_no_iso"

sig_path = "/home/grabanal/minitree/" * identifier * "_VVZ.arrow" 
bkg_path = "/home/grabanal/minitree/" * identifier * "_FULLBG.arrow" 
const sig = DataFrame(Arrow.Table(sig_path))
const bkg = DataFrame(Arrow.Table(bkg_path))
const sig_tp = Table(sig)
const bkg_tp = Table(bkg)

Table with 188 columns and 3000840 rows:
      lep2_iso_5  Zlep2_iso_4  Zlep2_mt  Zlep2_tight  Zlep1_iso_8  ⋯
    ┌───────────────────────────────────────────────────────────────
 1  │ true        true         26928.9   true         true         ⋯
 2  │ true        true         45493.2   true         true         ⋯
 3  │ true        true         37756.1   true         false        ⋯
 4  │ true        true         73870.1   true         true         ⋯
 5  │ true        false        20572.4   true         false        ⋯
 6  │ false       true         20948.9   false        false        ⋯
 7  │ true        false        22119.3   true         false        ⋯
 8  │ true        false        17199.8   true         false        ⋯
 9  │ false       false        14639.5   false        false        ⋯
 10 │ true        true         22735.3   true         false        ⋯
 11 │ true        true         44518.0   true         true         ⋯
 12 │ true        true         45271.1   true         true    

In [73]:
@macroexpand @. [1,2,3] + 3 + 6

:((+).([1, 2, 3], 3, 6))

In [4]:
names(bkg) |> println

["lep2_iso_5", "Zlep2_iso_4", "Zlep2_mt", "Zlep2_tight", "Zlep1_iso_8", "Wlep2_iso_6", "Zlep1_iso_3", "jet_m_2", "lep3_iso_1", "Zlep1_mt", "jet_m_3", "Zlep2_phi", "jet_m_4", "lep1_iso_3", "lep4_iso_4", "lep3_iso_7", "lep2_tight", "Zlep1_iso_10", "phi_1", "lep2_iso_7", "jet_phi_3", "Zlep2_pid", "Wlep1_pid", "lep4_iso_8", "jet_pt_3", "lep1_iso_9", "Zlep2_iso_5", "lep3_pid", "Zlep2_dphi", "lep3_tight", "m_4", "lep4_iso_10", "lep2_iso_8", "Zlep2_iso_7", "Zlep2_eta", "jet_pt_1", "Zlep1_iso_9", "Njet", "Wlep2_eta", "lep1_pid", "m_2", "Zlep2_iso_10", "Zlep2_iso_11", "lep3_iso_10", "Zlep1_tight", "lep4_iso_7", "lep2_iso_1", "Zlep2_pt", "jet_pt_2", "lep3_iso_3", "jet_eta_4", "lep3_iso_11", "Zlep1_iso_4", "lep2_iso_3", "Zlep2_iso_3", "lep1_iso_5", "other_mass", "lep1_iso_1", "Wlep2_iso_8", "lep4_tight", "eta_3", "jet_eta_2", "lep1_medium", "Wlep1_tight", "jet_phi_2", "Wlep2_iso_4", "Zlep2_iso_9", "lep4_pid", "lep4_iso_1", "Zlep1_iso_6", "lep3_iso_4", "Wlep2_iso_2", "Zlep1_iso_1", "Wlep2_phi", "c

In [24]:
function get_sweight(Wlep1_pid,Wlep2_pid,Zlep1_pid,Zlep2_pid,w1e,w1mu,w2e,w2mu,z1e,z1mu,z2e,z2mu,wgt)
    res = 0.0
    for idx in eachindex(w1e)
        wl1pid = abs(Wlep1_pid[idx])
        wl2pid = abs(Wlep2_pid[idx])
        zl1pid = abs(Zlep1_pid[idx])
        zl2pid = abs(Zlep2_pid[idx])
        flag =
        (wl1pid == 11 && w1e[idx] || wl1pid == 13 && w1mu[idx]) &&
        (wl2pid == 11 && w2e[idx] || wl2pid == 13 && w2mu[idx]) &&
        (zl1pid == 11 && z1e[idx] || zl1pid == 13 && z1mu[idx]) &&
        (zl2pid == 11 && z2e[idx] || zl2pid == 13 && z2mu[idx])
        res += flag ? wgt[idx] : 0.0
    end
    res
end

get_sweight (generic function with 3 methods)

In [8]:
function main()
    res = [Pair{Tuple{Int64, Int64}, Tuple{Float64, Float64}}[] for _ in 1:Threads.nthreads()]
    Threads.@threads for (isow,isoz) in collect(Iterators.product(3:3,7:7))
        col1 = Symbol(:Wlep1, :_iso_, isow)
        col2 = Symbol(:Wlep2, :_iso_, isow)
        col3 = Symbol(:Zlep1, :_iso_, isoz)
        col4 = Symbol(:Zlep2, :_iso_, isoz)
        iso_pic = (isow, isoz)
        bkg_mask = @. bkg[!, col1] & bkg[!, col2] & bkg[!, col3] & bkg[!, col4]
        sig_mask = @. sig[!, col1] & sig[!, col2] & sig[!, col3] & sig[!, col4]
        @views push!(res[Threads.threadid()], iso_pic => (sum(sig[sig_mask, :wgt]), sum(bkg[bkg_mask, :wgt])))
    end
    return res
end

main (generic function with 1 method)

In [31]:
function cons_sym(isoew,isomuw,isoez,isomuz)
    Symbol(:Wlep1, :_iso_, isoew),
    Symbol(:Wlep1, :_iso_, isomuw),
    Symbol(:Wlep2, :_iso_, isoew),
    Symbol(:Wlep2, :_iso_, isomuw),
    Symbol(:Zlep1, :_iso_, isoez),
    Symbol(:Zlep1, :_iso_, isomuz),
    Symbol(:Zlep2, :_iso_, isoez),
    Symbol(:Zlep2, :_iso_, isomuz)
end

cons_sym (generic function with 1 method)

In [57]:
function main2()
    res = [Pair{NTuple{4, Int64}, Tuple{Float64, Float64}}[] for _ in 1:Threads.nthreads()]
    Threads.@threads for (i1, i2, i3, i4) in collect(Iterators.product(1:11, 1:11, 1:11, 1:11))
        bkgbranches = getproperty.(Ref(bkg), cons_sym(i1, i2, i3, i4))
        sigbranches = getproperty.(Ref(sig), cons_sym(i1, i2, i3, i4))
        bkg_res = get_sweight(bkg.Wlep1_pid,bkg.Wlep2_pid,bkg.Zlep1_pid,bkg.Zlep2_pid, bkgbranches..., bkg.wgt)
        sig_res = get_sweight(sig.Wlep1_pid,sig.Wlep2_pid,sig.Zlep1_pid,sig.Zlep2_pid, sigbranches..., sig.wgt)
        push!(res[Threads.threadid()], (i1, i2, i3, i4) => (sig_res, bkg_res))
    end
    reduce(vcat, res)
end

main2 (generic function with 1 method)

In [58]:
res = @time main2();

152.859760 seconds (1.57 M allocations: 81.989 MiB, 0.00% compilation time)


In [67]:
res[3173]

(5, 3, 5, 3) => (42.09128250079475, 3716.7418330882238)

In [66]:
findmax(p->significance_and_error(p[2][1], 0.0, p[2][2], 0), res)

((0.6891191559288661, 0.0), 3173)

In [76]:
res[1]

10-element Vector{Pair{Tuple{Int64, Int64}, Tuple{Float64, Float64}}}:
 (1, 1) => (36.0729680507379, 3299.402839504189)
 (1, 2) => (36.59954774332903, 3305.435755091944)
 (2, 1) => (35.97286872093014, 3111.862004809716)
 (2, 2) => (36.603862381080596, 3131.306993720843)
 (3, 1) => (38.26525469410356, 3386.2759184491456)
 (3, 2) => (38.893506618115595, 3401.108562785788)
 (4, 1) => (35.102202711223335, 3012.7684730638784)
 (4, 2) => (35.69749827352121, 3027.312103394211)
 (5, 1) => (38.45563813893, 3488.189369550147)
 (5, 2) => (39.096207432547715, 3501.850097105305)

In [82]:
print("VVZ signal yield: ", sum(sig.wgt))
print("\n")
print("background yield: ", sum(bkg.wgt))

VVZ signal yield: 48.01371271488642
background yield: 5903.504636774685

In [156]:
sig.wgt.^2

171474-element Vector{Float64}:
 1.7024226169456366e-8
 1.3973349380121177e-8
 1.8683882414393683e-8
 1.9406012636190478e-8
 2.827700209690963e-8
 1.7980723600234236e-8
 2.3612699439598696e-8
 2.5510890881650877e-8
 2.5849005231009215e-11
 1.5410348153887595e-8
 1.772351260084546e-8
 1.0665142174745076e-8
 4.278912056761412e-8
 ⋮
 1.3432608033119304e-6
 2.0057445058728455e-7
 3.402794510656849e-7
 5.111978865287091e-7
 1.9431136165870995e-7
 2.416423177555663e-7
 5.866769486687698e-7
 2.6851748073755986e-7
 2.793446562076789e-7
 3.63613823621103e-7
 3.1747186547123067e-7
 2.2837431973441594e-7

In [65]:
function yield_and_error(df)
#     '''
#     input: 
#     df  : dataframe
    
#     output:
#     yield (significance, error)
#     '''
    Y  = -1
    dY = -1
    Y  = sum(df.wgt)
    dY = sqrt(sum(df.wgt.^2))
    return Y, dY
end

function significance_and_error(S, dS, B, dB)
#     '''
#     input: 
#     S  : signal
#     dS : signal error
#     B  : background
#     dB : background error
    
#     output:
#     tuple (significance, error)
#     '''
    # FAKE SIGNIFICANCE
#     significance = S/np.sqrt(B)
#     error        = np.sqrt( 1/B*(dS**2) + (S**2)/(4*B**3)*(dB**2) )
#     return significance, error
    
    # REAL SIGNIFICANCE
    # if a quantity is negative, return 0
    if ((S<=0) | (B<=0))
        return 0, 0
    end
    # calculate the significance and its error
    significance = sqrt( 2*((S + B)*log(1 + S/B) - S) )
    dSignifdS    = log(1 + S/B)/significance
    dSignifdB    = ( log(1 + S/B) - S/B )/significance
    error        = sqrt( (dSignifdS*dS)^2 + (dSignifdB*dB)^2 )
    # return tuple
    return significance, error
end

function add_in_quadrature_3( XdX, YdY, ZdZ )
#     '''
#     input: 
#     XdX : tuple (significance, error)
#     YdY : tuple (significance, error)
#     ZdZ : tuple (significance, error)
    
#     output:
#     tuple (significance, error) added in quadrature
#     '''
    # unpack the significances
    x, dx = XdX
    y, dy = YdY
    z, dz = ZdZ
    # calculate the addition in quadrature and its error
    quadrature_sum   = sqrt(x^2 + y^2 + z^2)
    A                = (x^2)*(dx^2) + (y^2)*(dy^2) + (z^2)*(dz^2)
    B                = (x^2) + (y^2) + (z^2)
    if (quadrature_sum == 0)
        return 0, 0
    end
    quadrature_error = sqrt( A / B )
    # return tuple
    return quadrature_sum, quadrature_error
end

function total_significance_and_error( df_sig, df_bg )
    
    m_list = []
    for sr in [0,1,2]
        S, dS = yield_and_error(df_sig[df_sig.SR .== sr])
        B, dB = yield_and_error( df_bg[ df_bg.SR .== sr])
        m_list.append(significance_and_error(S, dS, B, dB))
    end
    significance, error = add_in_quadrature_3(m_list[1], m_list[2], m_list[3])
    return significance, error
end

function print_total_significance_and_error( df_sig, df_bg )
    
    m_list = []
    for sr in [0,1,2]
        if (sr == 0)
            print("Region 4ℓ-SF-inZ:\n")
        end
        if (sr == 1)
            print("Region 4ℓ-SF-noZ:\n")
        end
        if (sr == 2)
            print("Region 4ℓ-DF:\n")
        end
        S, dS = yield_and_error(df_sig[df_sig.SR .== sr, :])
        @printf("--- S = %.2f ± %.2f \n", S, dS)
        B, dB = yield_and_error( df_bg[ df_bg.SR .== sr, :])
        @printf("--- B = %.2f ± %.2f \n", B, dB)
        significance, error = significance_and_error(S, dS, B, dB)
        @printf("--- significance = %.2f ± %.2f \n", significance, error)
        push!(m_list, (significance, error))
        print("\n")
    end
        
    significance, error = add_in_quadrature_3(m_list[1], m_list[2], m_list[3])
    @printf("Total significance in quadrature = %.2f ± %.2f \n", significance, error)
end

print_total_significance_and_error (generic function with 1 method)

In [158]:
yield_and_error(sig)

(48.01371271488642, 0.2432267729879009)

In [159]:
print_total_significance_and_error(sig, bkg)

Region 4ℓ-SF-inZ:
--- S = 14.39 ± 0.08 
--- B = 2916.97 ± 10.28 
--- significance = 0.27 ± 0.00 

Region 4ℓ-SF-noZ:
--- S = 15.41 ± 0.14 
--- B = 1723.12 ± 50.59 
--- significance = 0.37 ± 0.01 

Region 4ℓ-DF:
--- S = 18.21 ± 0.18 
--- B = 1263.41 ± 69.37 
--- significance = 0.51 ± 0.01 

Total significance in quadrature = 0.69 ± 0.01 


In [169]:
range(n_qualities)

LoadError: ArgumentError: Cannot construct range from arguments:
start = 2
step = nothing
stop = nothing
length = nothing
Try specifying more arguments.


In [183]:
for i in 0:n_qualities
    print(i)
end

0123

In [207]:
[zeros((n_qualities*n_iso)), zeros((n_qualities*n_iso)), zeros((n_qualities*n_iso))]

3-element Vector{Vector{Float64}}:
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [262]:
n_qualities = 9
n_iso_electrons = 6
n_iso_muons = 12
n_iso = n_iso_electrons*n_iso_muons

n_qualities = 2
n_iso_electrons = 2
n_iso_muons = 2
n_iso = n_iso_electrons*n_iso_muons

W_qual_iso_df = DataFrame()

for (n, df) in zip([0,1],[sig, bkg])
    
    if (n == 0)
        print("--- analyzing the signal\n")
    end
    if (n == 1)
        print("--- analyzing the background\n")
    end
        
#     y        = zeros((3,n_qualities*n_iso)) # for each SR
#     y_err    = zeros((3,n_qualities*n_iso)) # for each SR
    
    y = [[],[],[]]
    y_err = [[],[],[]]

    for SR in [0,1,2]
        
        @printf("--- Signal Region = %i\n", SR)
        counter = -1 # counter will go from 0 to 647
        sr_df = copy(df[df.SR .== SR, :])
        
        # Quality loop (LL, LM, LT, ML, MM, MT, TL, TM, TT)
        for quality in 0:(n_qualities-1)
            
            qual_df = copy(sr_df)
            # Check quality
            # LL
            if (quality == 0) 
            # LM
            elseif (quality == 1) 
                qual_df = qual_df[(abs.(qual_df.Wlep1_pid) .== 11) .| (qual_df.Wlep1_medium .== true), :] # medium muons 
                qual_df = qual_df[(abs.(qual_df.Wlep2_pid) .== 11) .| (qual_df.Wlep2_medium .== true), :] # medium muons
            # LT
            elseif  (quality == 2) 
                qual_df = qual_df[(abs.(qual_df.Wlep1_pid) .== 11) .| (qual_df.Wlep1_tight .== true), :] # tight muons
                qual_df = qual_df[(abs.(qual_df.Wlep2_pid) .== 11) .| (qual_df.Wlep2_tight .== true), :] # tight muons
            # ML
            elseif  (quality == 3) 
                qual_df = qual_df[(abs.(qual_df.Wlep1_pid) .== 13) .| (qual_df.Wlep1_medium .== true), :] # medium electrons
                qual_df = qual_df[(abs.(qual_df.Wlep2_pid) .== 13) .| (qual_df.Wlep2_medium .== true), :] # medium electrons
            # MM
            elseif  (quality == 4) 
                qual_df = qual_df[qual_df.Wlep1_medium .== true, :] # medium all
                qual_df = qual_df[qual_df.Wlep2_medium .== true, :] # medium all
            # MT
            elseif  (quality == 5) 
                qual_df = qual_df[(abs.(qual_df.Wlep1_pid) .== 13) .| (qual_df.Wlep1_medium .== true), :] # medium electrons
                qual_df = qual_df[(abs.(qual_df.Wlep1_pid) .== 11) .| (qual_df.Wlep1_tight .== true), :]  # tight muons
                qual_df = qual_df[(abs.(qual_df.Wlep2_pid) .== 13) .| (qual_df.Wlep2_medium .== true), :] # medium electrons
                qual_df = qual_df[(abs.(qual_df.Wlep2_pid) .== 11) .| (qual_df.Wlep2_tight .== true), :]  # tight muons
            # TL
            elseif (quality == 6) 
                qual_df = qual_df[(abs.(qual_df.Wlep1_pid) .== 13) .| (qual_df.Wlep1_tight .== true), :] # tight electrons
                qual_df = qual_df[(abs.(qual_df.Wlep2_pid) .== 13) .| (qual_df.Wlep2_tight .== true), :] # tight electrons
            # TM
            elseif (quality == 7)
                qual_df = qual_df[(abs.(qual_df.Wlep1_pid) .== 13) .| (qual_df.Wlep1_tight .== true), :]  # tight electrons
                qual_df = qual_df[(abs.(qual_df.Wlep1_pid) .== 11) .| (qual_df.Wlep1_medium .== true), :] # medium muons 
                qual_df = qual_df[(abs.(qual_df.Wlep2_pid) .== 13) .| (qual_df.Wlep2_tight .== true), :]  # tight electrons
                qual_df = qual_df[(abs.(qual_df.Wlep2_pid) .== 11) .| (qual_df.Wlep2_medium .== true), :] # medium muons
            # TT
            elseif (quality == 8)
                qual_df = qual_df[qual_df.Wlep1_tight .== true, :] # tight all
                qual_df = qual_df[qual_df.Wlep2_tight .== true, :] # tight all
            end

            # Isolation loop for electrons, 0 means no iso
            for i in 0:(n_iso_electrons-1)
                
                iso_e_df = copy(qual_df)
                if (i == 0)
                else
                    iso_e_df = iso_e_df[(abs.(iso_e_df.Wlep1_pid) .== 13) .| (iso_e_df[!, string("Wlep1_iso_", i)] .== true), :] # iso_i electrons
                    iso_e_df = iso_e_df[(abs.(iso_e_df.Wlep2_pid) .== 13) .| (iso_e_df[!, string("Wlep2_iso_", i)] .== true), :] # iso_i electrons
                end
                                                    
                # Isolation loop for muons, 0 means no iso
                for j in 0:(n_iso_muons-1)
                    
                    iso_mu_df = copy(iso_e_df)
                    if (j == 0)
                    else
                        iso_mu_df = iso_mu_df[(abs.(iso_mu_df.Wlep1_pid) .== 11) .| (iso_mu_df[!, string("Wlep1_iso_", j)] .== true ), :] # iso_j muons
                        iso_mu_df = iso_mu_df[(abs.(iso_mu_df.Wlep2_pid) .== 11) .| (iso_mu_df[!, string("Wlep2_iso_", j)] .== true ), :] # iso_j muons
                    end
                                                        
                    counter = counter + 1
                    @printf("counter = %i\n", counter)

#                     y[SR][counter]     = sum(iso_mu_df.wgt)
                    push!(y[SR+1], sum(iso_mu_df.wgt))
#                     y_err[SR][counter] = sum(iso_mu_df.wgt.^2)
                    push!(y_err[SR+1], sum(iso_mu_df.wgt.^2))
                end
            end
        end
    end

    y_err[1] = sqrt.(y_err[1])
    y_err[2] = sqrt.(y_err[2])
    y_err[3] = sqrt.(y_err[3])

    # Saving information:
    if (n == 0)
        W_qual_iso_df[!, :SF_inZ_signal]     = y[1]
        W_qual_iso_df[!, :SF_inZ_signal_err] = y_err[1]
        W_qual_iso_df[!, :SF_noZ_signal]     = y[2]
        W_qual_iso_df[!, :SF_noZ_signal_err] = y_err[2]
        W_qual_iso_df[!, :DF_signal]         = y[3]
        W_qual_iso_df[!, :DF_signal_err]     = y_err[3]
        W_qual_iso_df[!, :signal]            = y[1]+y[2]+y[3]
        W_qual_iso_df[!, :signal_err]        = sqrt(y_err[1]^2 + y_err[2]^2 + y_err[3]^2)
        
    elseif (n == 1)
        W_qual_iso_df[:SF_inZ_background]     = y[1]
        W_qual_iso_df[:SF_inZ_background_err] = y_err[1]
        W_qual_iso_df[:SF_noZ_background]     = y[2]
        W_qual_iso_df[:SF_noZ_background_err] = y_err[2]
        W_qual_iso_df[:DF_background]         = y[3]
        W_qual_iso_df[:DF_background_err]     = y_err[3]
        W_qual_iso_df[:background]            = y[1]+y[2]+y[3]
        W_qual_iso_df[:background_err]        = sqrt(y_err[1]^2 + y_err[2]^2 + y_err[3]^2)
    end 
        
    
end                          
                                            
# Calculating the significances
print("Calculating the significances\n")
significances = [[],[],[],[]] 
errors = [[],[],[],[]]

for choice in W_qual_iso_df.index:

    # SF-inZ
    significance, error = significance_and_error(W_qual_iso_df.at[choice,"SF-inZ-signal"], \
                                                 W_qual_iso_df.at[choice,"SF-inZ-signal-err"], \
                                                 W_qual_iso_df.at[choice,"SF-inZ-background"], \
                                                 W_qual_iso_df.at[choice,"SF-inZ-background-err"])
    significances[0].append(significance)
    errors[0].append(error)
    # SF-noZ
    significance, error = significance_and_error(W_qual_iso_df.at[choice,"SF-noZ-signal"], \
                                                 W_qual_iso_df.at[choice,"SF-noZ-signal-err"], \
                                                 W_qual_iso_df.at[choice,"SF-noZ-background"], \
                                                 W_qual_iso_df.at[choice,"SF-noZ-background-err"])
    significances[1].append(significance)
    errors[1].append(error)
    # DF
    significance, error = significance_and_error(W_qual_iso_df.at[choice,"DF-signal"], \
                                                 W_qual_iso_df.at[choice,"DF-signal-err"], \
                                                 W_qual_iso_df.at[choice,"DF-background"], \
                                                 W_qual_iso_df.at[choice,"DF-background-err"])
    significances[2].append(significance)
    errors[2].append(error)
    # Quadrature
    significance, error = add_in_quadrature_3((significances[1][-1], errors[1][-1]), 
                                              (significances[2][-1], errors[2][-1]),
                                              (significances[3][-1], errors[3][-1]),)
    significances[3].append(significance)
    errors[3].append(error)
    
W_qual_iso_df["SF-inZ-significance"]     = significances[1]
W_qual_iso_df["SF-inZ-significance-err"] = errors[1]
W_qual_iso_df["SF-noZ-significance"]     = significances[2]
W_qual_iso_df["SF-noZ-significance-err"] = errors[2]
W_qual_iso_df["DF-significance"]         = significances[3]
W_qual_iso_df["DF-significance-err"]     = errors[3]
W_qual_iso_df["significance"]            = significances[4]
W_qual_iso_df["significance-err"]        = errors[4]
print("--- Done!")

--- analyzing the signal
--- Signal Region = 0
counter = 0
counter = 1
counter = 2
counter = 3
counter = 4
counter = 5
counter = 6
counter = 7
--- Signal Region = 1
counter = 0
counter = 1
counter = 2
counter = 3
counter = 4
counter = 5
counter = 6
counter = 7
--- Signal Region = 2
counter = 0
counter = 1
counter = 2
counter = 3
counter = 4
counter = 5
counter = 6
counter = 7


LoadError: MethodError: no method matching ^(::Vector{Any}, ::Int64)
[0mClosest candidates are:
[0m  ^([91m::Union{AbstractChar, AbstractString}[39m, ::Integer) at strings/basic.jl:721
[0m  ^([91m::LinearAlgebra.UniformScaling[39m, ::Number) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.7/LinearAlgebra/src/uniformscaling.jl:318
[0m  ^([91m::LinearAlgebra.Hermitian[39m, ::Integer) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.7/LinearAlgebra/src/symmetric.jl:726
[0m  ...

In [256]:
sig[!, "SR"]

171474-element Arrow.Primitive{Int32, Vector{Int32}}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 1
 1
 ⋮
 1
 2
 0
 2
 1
 2
 1
 2
 1
 1
 2
 1

In [249]:
x = sig[ (sig.SR .== 0) .| (sig.Wlep1_tight .== true) , :]

,lep2_iso_5,Zlep2_iso_4,Zlep2_mt,Zlep2_tight,Zlep1_iso_8,Wlep2_iso_6,Zlep1_iso_3
,Bool,Bool,Float32,Bool,Bool,Bool,Bool
1,1,1,22867.6,1,0,1,1
2,1,0,25179.8,1,0,0,1
3,1,1,42508.4,1,0,0,1
4,1,1,22017.4,0,1,0,1
5,1,1,50298.2,1,0,0,0
6,1,1,36787.7,1,1,1,1
7,1,1,20295.6,1,1,1,1
8,1,1,35853.7,1,1,0,1
9,1,1,39495.5,1,1,1,1


In [192]:
n_qualities*n_iso

648

In [201]:
str = string("Wlep1_iso_",10)

"Wlep1_iso_10"